In [1]:
from langchain_community.tools import WikipediaQueryRun, ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [2]:
# Tool for wikipedia
wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)
wiki.name

'wikipedia'

In [3]:
# Tool for arxiv
arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)
arxiv.name

'arxiv'

In [4]:
tools = [wiki, arxiv]

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectorstoredb =  FAISS.from_documents(documents,embeddings)
retriever = vectorstoredb.as_retriever()
retriever


/home/priteshs/.local/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x79dbb8111b80>, search_kwargs={})

In [7]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool = create_retriever_tool(retriever,'langsmith-search','search any information about langchain')
retriver_tool.name

tools.append(retriver_tool)

In [8]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/priteshs/anaconda3/envs/learning_llm_udemy/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='search any information about langchain', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x79dcee299e40>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x

In [9]:
## Run all this tools with Agents and LLM Models

## Tools, LLM-->AgentExecutor
from langchain_groq import ChatGroq
groq_api_key = os.getenv('GROQ_API_KEY')
llm = ChatGroq(model='Gemma2-9b-It', api_key=groq_api_key)

In [10]:
## Prompt Template
from langchain import hub
prompt = hub.pull('hwchase17/openai-functions-agent')
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [11]:
## Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [12]:
## Agent Executor 
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [13]:
agent_executor.invoke({'input':'Tell me about langsmith'})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

4. Log your first trace​
Tracing to LangSmith for LangChain usersThere is no need to use the LangSmith SDK directly if your application is built entirely on LangChain (either Python and JS).We've outlined a tracing guide specifically for LangChain users here.
We provide multiple ways to log traces to LangSmith. Below, we'll highlight
how to use traceable. See more on the Annotate code for tracing page.

PythonTypeScriptimport openaifrom langsmith.wrappers import wrap_openaifrom langsmith import traceable# Auto-trace LLM calls in-contextclient = wrap_openai(openai.Client())@traceable # Auto-trace this functiondef pipeline(user_input: str):    result = client.chat.completions.create(        messages=[{"role": "user", "content": user_input}],        model="gpt-4o-mini"    )    return result.choices[0].message.contentpipeline("Hello, world!")# O

{'input': 'Tell me about langsmith',
 'output': 'LangSmith is a tool for building and evaluating language models. It is open-source and can be used with both Python and JavaScript. LangSmith provides a number of features that make it easy to get started with building and evaluating language models, including:\n\n- A simple API for logging traces of language model interactions.\n- A set of tools for evaluating the performance of language models.\n- A community forum and documentation for getting help and support.'}

In [14]:
agent_executor.invoke({"input":"What is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks witMachine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks with minimal human intervention. 




> Finished chain.


{'input': 'What is machine learning',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks with minimal human intervention. \n\n\n'}

In [15]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cThe paper 1706.03762, titled "Attention Is All You Need", is about a new neural network architecture called the Transformer, which relies entirely on attention mechanisms to compute representations of input sequences. 


> Finished chain.


{'input': "What's the paper 1706.03762 about?",
 'output': 'The paper 1706.03762, titled "Attention Is All You Need", is about a new neural network architecture called the Transformer, which relies entirely on attention mechanisms to compute representations of input sequences. \n'}

In [16]:
agent_executor.invoke({"input":"What is AI transformer by arxiv?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'AI transformer'}`


Published: 2024-08-26
Title: AI Thinking: A framework for rethinking artificial intelligence in practice
Authors: Denis Newman-Griffis
Summary: Artificial intelligence is transforming the way we work with information
across disciplines and practical It looks like the tool returned a result about AI Thinking, not specifically about "AI transformer" on Arxiv.  
 
Would you like me to try searching for "AI transformer" again on Arxiv? 



> Finished chain.


{'input': 'What is AI transformer by arxiv?',
 'output': 'It looks like the tool returned a result about AI Thinking, not specifically about "AI transformer" on Arxiv.  \n \nWould you like me to try searching for "AI transformer" again on Arxiv? \n\n'}

In [18]:
agent_executor.invoke({"input":"what is best way to learn AI?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'best way to learn AI'}`


Page: Neuro-symbolic AI
Summary: Neuro-symbolic AI is a type of artificial intelligence that integrates neural and symbolic AI architectures to address the weaknesses of each, providing a robust AI capable of reasoning, learning, and cognitive modeliThat's a great area to learn about! 

Neuro-symbolic AI combines the strengths of both symbolic and neural approaches, aiming to create AI systems that can not only learn patterns from data but also reason and explain their decisions. 

To learn about this field, you could explore:

* **Online Courses:** Platforms like Coursera, edX, and Udacity offer courses on AI, machine learning, and neuro-symbolic AI.
* **Books:**  Look for books on neuro-symbolic AI, cognitive architectures, and the integration of symbolic and neural methods.
* **Research Papers:** Read recent publications in top AI conferences and journals to stay up-to-date on t

{'input': 'what is best way to learn AI?',
 'output': "That's a great area to learn about! \n\nNeuro-symbolic AI combines the strengths of both symbolic and neural approaches, aiming to create AI systems that can not only learn patterns from data but also reason and explain their decisions. \n\nTo learn about this field, you could explore:\n\n* **Online Courses:** Platforms like Coursera, edX, and Udacity offer courses on AI, machine learning, and neuro-symbolic AI.\n* **Books:**  Look for books on neuro-symbolic AI, cognitive architectures, and the integration of symbolic and neural methods.\n* **Research Papers:** Read recent publications in top AI conferences and journals to stay up-to-date on the latest advancements.\n* **Open-Source Projects:** Get involved in open-source projects that develop neuro-symbolic AI systems.\n\n\n\nLet me know if you have any other questions about neuro-symbolic AI or other AI learning resources. \n\n"}